In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -r '/content/drive/MyDrive/requirements_14_nov.txt'

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Traceback (most recent call last):
  File "/usr/local/bin/huggingface

In [9]:
from datasets import load_dataset
dataset = load_dataset('gh1407/inference_197_rows')
print(dataset)

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['response'],
        num_rows: 197
    })
})


In [45]:

import re

train_ds = dataset['train']
# output dictionary. keep track of the original phrase, the original cause and effect,
# and the summarized cause and effect
out_dict = {'original_string': [], 'cause_split': [], 'effect_split': [], 'cause_summarized': [], 'effect_summarized': []}
for row in train_ds:
  response = row['response']
  out_dict['original_string'].append(response)
  splitted = re.split(r'\[cause\]|-> \[effect\]',response)
  splitted = splitted[1:]
  if len(splitted)<2:
    continue
  inds = range(0, len(splitted), 2)
  causes = [splitted[i] for i in inds]
  effects = [splitted[i+1] for i in inds if i+1<=len(splitted)-1]
  out_dict['cause_split'].extend(causes)
  out_dict['effect_split'].extend(effects)



In [46]:
print(out_dict['original_string'][0])

<s> Below is an excerpt from a news article. Return the cause and effect phrases.

### Text:
Attorney General Jeff Sessions (Photo: Alex Wong, Getty Images)
WASHINGTON — Attorney General Jeff Sessions acknowledged Monday that the FBI's information system failed to preserve five months of text messages between two bureau officials who had disparaged then-candidate Donald Trump during the 2016 election.
The discovery of the communications earlier this year prompted the removal of Peter Strzok, a senior counter-intelligence agent, from the staff of Russia special counsel Robert Mueller.
Strzok had been communicating by text message for months with colleague Lisa Page, who also had been assigned to Mueller's team, but had returned her duties at the FBI before the text messages were found.
The Justice Department turned over a tranche of communications between the two officials to Congress last month covering a period between August 2015 to December 2016. In those contacts, Strzok, who also 

In [47]:
print(out_dict['cause_split'][0:4])

[" Attorney General Jeff Sessions acknowledged Monday that the FBI's information system failed to preserve five months of text messages between two bureau officials who had disparaged then-candidate Donald Trump during the 2016 election ", ' The discovery of the communications earlier this year prompted the removal of Peter Strzok, a senior counter-intelligence agent, from the staff of Russia special counsel Robert Mueller ', ' Threatening phone calls ', ' Threatening phone calls ']


In [12]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.7 MB/s eta 0:00:00


In [19]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [71]:
from langchain import PromptTemplate,  LLMChain

template = """
              Write a concise summary of the subject in the following text.
              Limit your summary to three words or less.
              TEXT: {text}
              SUMMARY:
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [32]:
text = """As part of Meta’s commitment to open science, today we are publicly
releasing LLaMA (Large Language Model Meta AI), a state-of-the-art foundational
large language model designed to help researchers advance their work in this
subfield of AI. Smaller, more performant models such as LLaMA enable others
in the research community who don’t have access to large amounts of
infrastructure to study these models, further democratizing access in this
important, fast-changing field.Training smaller foundation models like LLaMA
is desirable in the large language model space because it requires far less
computing power and resources to test new approaches, validate others’ work,
and explore new use cases. Foundation models train on a large set of unlabeled
data, which makes them ideal for fine-tuning for a variety of tasks.
We are making LLaMA available at several sizes (7B, 13B, 33B, and 65B
parameters) and also sharing a LLaMA model card that details how we built
the model in keeping with our approach to Responsible AI practices.
Over the last year, large language models — natural language processing (NLP)
systems with billions of parameters — have shown new capabilities to generate
creative text, solve mathematical theorems, predict protein structures,
answer reading comprehension questions, and more. They are one of the clearest
cases of the substantial potential benefits AI can offer at scale to billions
of people"""


In [33]:
print(llm_chain.run(text))

 Large language model LLaMA released by Meta to democratize access in AI field.


### experimentation

In [72]:
out_dict['cause_summarized'] = []
out_dict['effect_summarized'] = []
for phrase in out_dict['cause_split'][0:6]:
  # phrase = re.sub(r'[^a-zA-Z .]+', '', phrase)
  print(f"phrase: {phrase}")
  result = llm_chain.run(phrase)
  result = re.sub(r'[^a-zA-Z .]+', '', result)
  result = re.sub(r'SUMMARY|summarytext', '', result)
  # print(f"summary: {result}")
  result_cleaned = ' '.join(result.split()[:4])
  print(f"result_cleaned: {result_cleaned}")

phrase:  Attorney General Jeff Sessions acknowledged Monday that the FBI's information system failed to preserve five months of text messages between two bureau officials who had disparaged then-candidate Donald Trump during the 2016 election 


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


result_cleaned: FBI failed to preserve
phrase:  The discovery of the communications earlier this year prompted the removal of Peter Strzok, a senior counter-intelligence agent, from the staff of Russia special counsel Robert Mueller 


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


result_cleaned: Agent removed from Mueller
phrase:  Threatening phone calls 


...

result_cleaned: Caller menaces threats harassment.
phrase:  False information about a hostage situation 


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


result_cleaned: Hostage situation hoax If
phrase:  Bomb threats against Jewish community centers 


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


result_cleaned: Threats against Jewish centers


### run for 50 examples

In [73]:
out_dict['cause_summarized'] = []
out_dict['effect_summarized'] = []
i = 0
for phrase in out_dict['cause_split'][0:51]:
  print(f"i: {i}")
  i+=1
  result = llm_chain.run(phrase)
  result = re.sub(r'[^a-zA-Z .]+', '', result)
  result = re.sub(r'SUMMARY|summarytext', '', result)
  # print(f"summary: {result}")
  result_cleaned = ' '.join(result.split()[:4])
  out_dict['cause_summarized'].append(result_cleaned)

i: 0


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


i: 1


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


i: 2


...

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


i: 49


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


i: 50


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [75]:
print(out_dict['cause_summarized'][0:25])

['FBI lost texts.', 'Strzok removed from Mueller', 'Threats over phone B.', 'Caller threatens violence', 'False info hostage A.', 'Bomb threats Against Jewish', 'Threats against Jewish community', 'Threats against Jewish centers', 'Trump sanctions Russia New', 'U.S. imposes sanctions on', 'Tough on Russia', 'Trump not tough enough', 'USUN relations strained Ambassador', 'ExMongolian finance minister received', 'Rio Tinto not guilty', 'Oyu Tolgoi payment not', 'Swiss prosecutors seize .', 'Investigating corruption cases involving', 'Gains extended EXTENDED GAINS', 'Fed chair nominees economic', 'Asian markets surge Continued', 'Fed rate hike clues', 'Fed on vergeAnswer Fed', 'Pilot for Trump to', 'Dunkin Donuts flight experience']


In [76]:
i = 0
for phrase in out_dict['effect_split'][0:51]:
  print(f"i: {i}")
  i+=1
  result = llm_chain.run(phrase)
  result = re.sub(r'[^a-zA-Z .]+', '', result)
  result = re.sub(r'SUMMARY|summarytext', '', result)
  # print(f"summary: {result}")
  result_cleaned = ' '.join(result.split()[:4])
  out_dict['effect_summarized'].append(result_cleaned)

i: 0


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


i: 1


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


i: 2


...

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


i: 49


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


i: 50


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [78]:
from datasets import Dataset
ds = Dataset.from_dict(out_dict)
ds.push_to_hub('gh1407/inference_pairs_clustered')

ArrowInvalid: ignored

In [85]:
print(len(out_dict['original_string']))
out_dict['original_string'] = out_dict['original_string'] + [''] * (1302 - len(out_dict['original_string']))
print(len(out_dict['original_string']))

197
1302


In [81]:
print(len(out_dict['cause_split']))

1302


In [87]:
print(len(out_dict['effect_split']))
out_dict['effect_split'] = out_dict['effect_split'] + [''] * (1302 - len(out_dict['effect_split']))
print(len(out_dict['effect_split']))

1259
1302


In [89]:
print(len(out_dict['cause_summarized']))
out_dict['cause_summarized'] = out_dict['cause_summarized'] + [''] * (1302 - len(out_dict['cause_summarized']))
print(len(out_dict['cause_summarized']))

51
1302


In [90]:
print(len(out_dict['effect_summarized']))
out_dict['effect_summarized'] = out_dict['effect_summarized'] + [''] * (1302 - len(out_dict['effect_summarized']))
print(len(out_dict['effect_summarized']))

51
1302


In [91]:
from datasets import Dataset
ds = Dataset.from_dict(out_dict)
ds.push_to_hub('gh1407/inference_pairs_clustered')

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]